In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
from pathlib import Path

from src.data.raw.cleaner import clean_descriptions

[nltk_data] Downloading package punkt to /home/llazzari/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/llazzari/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to /home/llazzari/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


In [16]:
old_train_data = pd.read_csv(Path.cwd() / "database" / "ml" / "train_expenses_data.csv", index_col="id")
old_train_data.dropna(inplace=True)
old_train_data.head()

,description,amount,category,subcategory,year,month,recurrent,bank,cleaned_description
id,,,,,,,,,
0,BRASCON,110.08,Gastos Pessoais,Vestuário,2024,3,Não,C6,brascon
1,FARMACIA SAO JOAO,34.85,Saúde,Medicações,2024,3,Não,C6,farmacia sao joao
2,HAVAN PELOTAS,86.96,Moradia,Compras para casa,2024,3,Não,C6,havan
3,MARIAFLOR,99.80,Moradia,Compras para casa,2024,3,Não,C6,mariaflor
4,993 SHOPPING PELOTAS,49.98,Gastos Pessoais,Vestuário,2024,3,Não,C6,shopping


In [17]:
new_lucas_data = pd.read_csv(Path.cwd() / "database" / "lucas_expenses.csv", index_col="id")
new_lucas_data.head()

,date,description,amount,category,subcategory,year,month,recurrent,cleaned_description,bank
id,,,,,,,,,,
150,2024-09-25T00:00:00,Liquidacao B3 * 23/09/2024,128.58,Poupança/Dívidas/Investimentos,Investimentos,2024,9,Não,liquidacao b3 23 09 2024,Inter
151,2024-09-16T00:00:00,Inter Pre 10gb Mensal,30.00,Utilidades,Celular/Telefone Móvel,2024,9,Sim,inter pre 10gb mensal,Inter
152,2024-09-09T00:00:00,Fatura cartão Inter,708.77,Diversos,Despesas Diversas,2024,9,Não,fatura inter,Inter
153,2024-09-06T00:00:00,Popos Dog Hotel,60.00,Diversos,Animais de estimação,2024,9,Não,popos dog hotel,Inter
154,2024-09-05T00:00:00,CIA ESTADUAL DE DIST,221.43,Utilidades,Eletricidade,2024,9,Sim,cia estadual dist,Inter


In [18]:
new_nic_data = pd.read_csv(Path.cwd() / "database" / "nic_expenses.csv", index_col="id")
new_nic_data.head()

,date,description,amount,category,subcategory,year,month,recurrent,bank,cleaned_description
id,,,,,,,,,,
413,2023-12-27T00:00:00,MARIAFLOR,99.80,Moradia,Compras para casa,2024,5,Não,C6,mariaflor
414,2023-12-28T00:00:00,VLB COMERCIO DE ARTIGO,56.50,Gastos Pessoais,Vestuário,2024,5,Não,C6,vlb comercio artigo
415,2023-11-21T00:00:00,VINDI *INVESTIDOR10,19.90,Gastos Pessoais,Assinaturas,2024,5,Sim,C6,vindi investidor10
416,2024-01-10T00:00:00,ZP*PAGTODOS AMOR,63.53,Saúde,Despesas Médicas,2024,5,Não,C6,zp pagtodos amor
417,2024-01-20T00:00:00,HAVAN PELOTAS,73.97,Moradia,Compras para casa,2024,5,Não,C6,havan


In [20]:
expenses = pd.concat([old_train_data, new_lucas_data, new_nic_data])
expenses.sort_values(by=['year', 'month'], inplace=True)
expenses.drop(columns=['date'], inplace=True)
expenses['recurrent'].replace({'Sim': 1, 'Não': 0}, inplace=True)
expenses.reset_index(inplace=True)
expenses.head()

,id,description,amount,category,subcategory,year,month,recurrent,bank,cleaned_description
0,423,Aplicação RDB,2000.00,Poupança/Dívidas/Investimentos,Investimentos,2023,1,0,Nubank,aplicacao rdb
1,1077,Aplicação RDB,2000.00,Poupança/Dívidas/Investimentos,Investimentos,2023,1,0,Nubank,aplicacao rdb
2,1078,Fatura cartão Inter,491.11,Diversos,Despesas Diversas,2023,1,1,Inter,fatura inter
3,1079,Pag*alohapoke Pelotas Bra,101.00,Alimentação,Comer Fora,2023,1,0,Inter,alohapoke
4,1080,Inter Pre 4gb Mensal Redes Sociais,30.00,Utilidades,Celular/Telefone Móvel,2023,1,1,Inter,inter pre 4gb mensal redes sociais


In [6]:
expenses.loc[expenses['cleaned_description'].isna()][['description']]

,description
800,Fina Flor
801,Pg *Intershop Intersho 6/10
802,Styllo 10/10
803,Comercial Zaffari Ltda
804,Carlos Luiz Rodeghiero 169998860
...,...
1997,CREDITO OPERACAO B3-26/09/2024
1998,PGTO DE BOLETO-Boleto
1999,TRANSF ENVIADA PIX
2000,TRANSF ENVIADA PIX


In [7]:
expenses['cleaned_description'].fillna(expenses['description'], inplace=True)
expenses_cleaned: pd.DataFrame = clean_descriptions([], expenses)
expenses_cleaned.isna().sum()

index                  0
description            0
amount                 0
category               0
subcategory            0
year                   0
month                  0
recurrent              0
bank                   0
cleaned_description    0
dtype: int64

In [9]:
expenses_cleaned.head(5)

,index,description,amount,category,subcategory,year,month,recurrent,bank,cleaned_description
0,1076,Aplicação RDB,2000.00,Poupança/Dívidas/Investimentos,Investimentos,2023,1,0,Nubank,aplicaca rdb
1,1077,Aplicação RDB,2000.00,Poupança/Dívidas/Investimentos,Investimentos,2023,1,0,Nubank,aplicaca rdb
2,1078,Fatura cartão Inter,491.11,Diversos,Despesas Diversas,2023,1,1,Inter,fatur int
3,1079,Pag*alohapoke Pelotas Bra,101.00,Alimentação,Comer Fora,2023,1,0,Inter,alohapok
4,1080,Inter Pre 4gb Mensal Redes Sociais,30.00,Utilidades,Celular/Telefone Móvel,2023,1,1,Inter,int pre


In [16]:
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

df = expenses_cleaned.copy()

models = {
    "RandomForest": RandomForestClassifier(),
    "LogisticRegression": LogisticRegression(),
    "SVM": SVC(),
    "MLPClassifier": MLPClassifier(max_iter=500)
}

def run_models(models, X_train, X_test, y_train, y_test):
    results = {}
    
    for name, model in models.items():
        pipeline = Pipeline([
            ('tfidf', TfidfVectorizer()),  
            ('model', model)
        ])
        
        pipeline.fit(X_train, y_train)

        y_pred = pipeline.predict(X_test)
        
        accuracy = accuracy_score(y_test, y_pred)
        report = classification_report(y_test, y_pred)
        
        results[name] = {
            "accuracy": accuracy,
            "classification_report": report
        }
        print(f"Modelo: {name}")
        print(f"Acurácia: {accuracy*100:.2f}%")
        print(report)
    
    return results

X = df['cleaned_description']
y = df['subcategory']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

results = run_models(models, X_train, X_test, y_train, y_test)

Modelo: RandomForest
Acurácia: 91.01%
                                   precision    recall  f1-score   support

                       Acomodação       0.00      0.00      0.00         2
             Animais de estimação       1.00      1.00      1.00        22
                      Assinaturas       0.86      1.00      0.92         6
                       Bares/Pubs       1.00      1.00      1.00        10
       Beleza e Cuidados Pessoais       0.83      1.00      0.91        15
           Celular/Telefone Móvel       1.00      1.00      1.00        16
                       Comer Fora       0.97      0.71      0.82        42
        Compartilhamento de Carro       1.00      1.00      1.00         2
                Compras para casa       0.95      0.98      0.97       108
                Despesas Diversas       0.74      0.89      0.81        95
                 Despesas Médicas       1.00      1.00      1.00         4
                     Eletricidade       1.00      1.00      1

## With keywords as features

In [9]:
keyword_mapping: dict[str, list[str]] = {
    # Moradia
    "Hipoteca/Aluguel": ["hipoteca", "aluguel", "locação", "imóvel"],
    "Manutenção residencial": ["manutenção", "reparos", "conserto", "residencial"],
    "Impostos sobre a propriedade": ["iptu", "imposto", "propriedade"],
    "Compras para a casa": ["móveis", "decoração", "eletrodoméstico", "cama", "mesa", "banho"],
    
    # Transporte
    "Gasolina": ["gasolina", "combustível", "posto", "etanol", "diesel"],
    "Seguro do automóvel": ["seguro automóvel", "proteção veículo", "apólice automóvel"],
    "Manutenção do automóvel": ["revisão", "oficina", "mecânico", "troca de óleo", "pneus"],
    "Impostos e taxas do automóvel": ["ipva", "licenciamento", "detran", "multas"],
    "Transporte público": ["ônibus", "metrô", "bilhete", "passagem"],
    "Compartilhamento de carros": ["uber", "99", "cabify", "carona"],

    # Utilidades
    "Eletricidade": ["energia", "luz", "eletricidade", "enel", "ceee", "equatorial"],
    "Água": ["água", "sanasa", "sabesp", "sanep"],
    "Tarifas bancárias": ["tarifa bancária", "taxa banco", "cesta serviços"],
    "Celular": ["celular", "telefone", "tim", "claro", "vivo", "oi"],
    "Internet": ["internet", "wi-fi", "banda larga", "net", "fibra óptica"],

    # Alimentação
    "Mantimentos": ["supermercado", "mercado", "compras", "alimentos", "mantimentos", "macro atacado"],
    "Comer fora": ["restaurante", "lanchonete", "pizzaria", "bar", "café"],

    # Finanças
    "Poupança": ["poupança", "depósito", "reserva", "fundo"],
    "Dívidas": ["empréstimo", "financiamento", "dívida", "parcelamento"],
    "Investimentos": ["investimento", "ações", "bolsa", "renda fixa", "tesouro direto", "cdb", "rdb"],

    # Saúde
    "Plano de saúde": ["plano de saúde", "unimed", "bradesco saúde"],
    "Seguro saúde": ["seguro saúde", "apólice saúde", "cobertura médica"],
    "Despesas médicas": ["consultas", "exames", "tratamentos", "procedimentos médicos"],
    "Fitness": ["academia", "exercícios", "ginástica", "crossfit", "pilates"],
    "Medicações": ["remédio", "farmácia", "medicamento", "genérico"],
    "Suplementos": ["suplemento", "vitamina", "proteína", "whey"],

    # Pessoais
    "Vestuário": ["roupa", "calçado", "vestido", "camiseta", "sapato"],
    "Beleza e cuidados pessoais": ["cabeleireiro", "manicure", "barbearia", "cosméticos"],
    "Hobbies e recreações": ["livros", "videogame", "instrumentos", "brinquedos"],
    "Assinaturas": ["netflix", "spotify", "amazon prime", "hbo", "globo play", "investidor10"],
    "Estudos": ["curso", "escola", "faculdade", "matrícula", "aula", "seminário"],

    # Lazer
    "Filmes e séries": ["cinema", "filme", "série", "tv", "streaming"],
    "Eventos e atividades": ["evento", "concerto", "show", "festa", "balada"],
    "Bares e pubs": ["bar", "pub", "bebida", "cerveja", "whisky", "vinho"],

    # Viagens
    "Locomoção em viagens": ["passagem", "avião", "translado"],
    "Hospedagem": ["hotel", "airbnb", "pousada", "resort"],
    "Outros custos com viagens": ["souvenir", "passeios", "excursão", "tour", "guia"],

    # Diversos
    "Presentes e doações": ["presente", "doação", "aniversário", "casamento"],
    "Despesas diversas": ["diversos", "outros", "aleatórios", "extra"],
    "Férias": ["férias", "viagem", "descanso"],
    "Pets": ["pet", "animal", "ração", "veterinário", "banho e tosa", "vet"]
}

In [10]:
def map_keywords(description: str, keyword_mapping: dict) -> dict:
    """
    Maps keywords to relevant categories.
    
    Parameters:
    - description: Cleaned description from the dataframe.
    - keyword_mapping: A dictionary with subcategories as keys and lists of keywords as values.

    Returns:
    - A dictionary where the key is the subcategory and the value is 1 (if a keyword is present) or 0.
    """
    mapped_features = {category: 0 for category in keyword_mapping}
    
    for category, keywords in keyword_mapping.items():
        for keyword in keywords:
            if keyword in description:
                mapped_features[category] = 1
                break  # Se encontrar uma palavra, pode parar para essa categoria

    return mapped_features

In [11]:
def add_keyword_features(df: pd.DataFrame, keyword_mapping: dict) -> pd.DataFrame:
    """
    Adds keyword-based features to the DataFrame.
    
    Parameters:
    - df: DataFrame containing the cleaned descriptions.
    - keyword_mapping: A dictionary with subcategories as keys and lists of keywords as values.

    Returns:
    - The updated DataFrame with new columns for each subcategory.
    """
    keyword_features = df['cleaned_description'].apply(
        lambda desc: map_keywords(desc, keyword_mapping)
    )
    
    # Converte as features em DataFrame e adiciona ao original
    keyword_df = pd.DataFrame(keyword_features.tolist(), index=df.index)
    
    # Concatena o DataFrame original com as novas colunas
    df = pd.concat([df, keyword_df], axis=1)
    
    return df


In [14]:
from nltk.stem import RSLPStemmer

# Inicializando o stemmer
stemmer = RSLPStemmer()

def stem_keywords(keyword_mapping):
    """Aplica o stemming às palavras-chave no mapeamento."""
    stemmed_mapping = {}
    for subcategory, keywords in keyword_mapping.items():
        stemmed_keywords = [stemmer.stem(keyword) for keyword in keywords]
        stemmed_mapping[subcategory] = stemmed_keywords
    return stemmed_mapping

# Aplicar o stemming no keyword_mapping
stemmed_keyword_mapping = stem_keywords(keyword_mapping)

# Quando for realizar o mapeamento, o stemmed_keyword_mapping já estará pronto
df = add_keyword_features(expenses_cleaned, keyword_mapping)


In [15]:
# Definindo as colunas que serão usadas como features (incluindo cleaned_description)
feature_columns = ["cleaned_description"] + list(keyword_mapping.keys())

# Separando as features e o target
X = df[feature_columns]
y = df['subcategory']  # ou qual for a coluna com a subcategoria que você quer prever

# Dividir em treino e teste
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# vectorizer = CountVectorizer()
vectorizer = TfidfVectorizer()
# Aplicar o vectorizer apenas na coluna 'cleaned_description'
X_train_desc = vectorizer.fit_transform(X_train['cleaned_description'])
X_test_desc = vectorizer.transform(X_test['cleaned_description'])

# Remover a coluna 'cleaned_description' para trabalhar com as demais features (as colunas do keyword_mapping)
X_train_keywords = X_train.drop(columns=['cleaned_description'])
X_test_keywords = X_test.drop(columns=['cleaned_description'])

# Concatenar a matriz esparsa das descrições com as features binárias das palavras-chave
from scipy.sparse import hstack

X_train_combined = hstack([X_train_desc, X_train_keywords])
X_test_combined = hstack([X_test_desc, X_test_keywords])

# Treinando o modelo
model = RandomForestClassifier(random_state=42, n_estimators=100, max_depth=None, min_samples_leaf=1, min_samples_split=5)
model.fit(X_train_combined, y_train)

# Fazendo previsões e avaliando o modelo
predictions = model.predict(X_test_combined)
accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions)

print(f"Accuracy: {accuracy*100:.2f}%")


Accuracy: 91.18%
